http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

 sklearn.metrics.f1_score(y_true, y_pred, labels=None, pos_label=1, average=’binary’, sample_weight=None)[source]¶

In [1]:
# Import libraries
import os
import sys

from sqlalchemy import create_engine
import sqlite3

#import numpy as np
import pandas as pd
from re import sub

# cpu_count returns the number of CPUs in the system.
from multiprocessing import cpu_count

# Import from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Import models from sklearn
from sklearn.dummy import DummyClassifier

# Import XGBClassifier
from xgboost.sklearn import XGBClassifier

# Modify notebook settings
#%matplotlib inline
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [2]:
# Create a engine
engine = create_engine('sqlite:///raw_instacart_copy.db', echo=False)

## Read in data

In [112]:
sql_query = \
"""
SELECT *
FROM final_data_table_slim
"""

df = pd.read_sql_query(sql=sql_query, con=engine)
df.head()

,y,total_buy_n5,order_ratio_by_chance_n5
0,1,5,5
1,1,5,5
2,0,0,0
3,0,5,5
4,1,2,4


In [113]:
# Extract X and y from df
X = df.drop('y', axis=1).values
#y = df[['y']].values
y = df['y'].values

# Train test split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=42)

In [114]:
print('{:,}'.format(len(y)))
print('{:,}'.format(len(y_train)))
print('{:,}'.format(len(y_test)))

13,307,953
9,315,567
3,992,386


# Create Pipeline

In [41]:
# Create pipeline
pipe = Pipeline([('classifier', DummyClassifier(strategy='most_frequent',
                                                random_state=42))])

# Create parameter grid
param_grid = [
    {'classifier': [XGBClassifier(objective='binary:logistic', n_estimators=100)], 
     'classifier__n_estimators': [100],
     'classifier__learning_rate': [0.01],
     'classifier__gamma': [0.01],
     'classifier__max_depth': [3],
     'classifier__subsample': [0.8],
     'classifier__reg_lambda': [0.1],
     'classifier__reg_alpha': [0.1]}]

In [46]:
# Set the number of cores to be used
cores_used = cpu_count() - 1
cores_used
cores_used = 1

In [97]:
# Set verbosity
verbosity = 1

# Execute Grid search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='roc_auc',
                    verbose=verbosity, n_jobs=cores_used)

grid.fit(X_train, y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  9.0min finished


Best params:
{'classifier': XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.01, learning_rate=0.01, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0.1, reg_lambda=0.1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8), 'classifier__gamma': 0.01, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 3, 'classifier__n_estimators': 100, 'classifier__reg_alpha': 0.1, 'classifier__reg_lambda': 0.1, 'classifier__subsample': 0.8}

Best cross-validation score: 0.78
